In [1]:
import sys; sys.path.append("../../")

import numpy as np
import pandas as pd
import xarray as xr

from melodies_monet import driver

In [2]:
an = driver.analysis()
an.control = "control_idealized.yaml"
an

analysis(
    control='control_idealized.yaml',
    control_dict=None,
    models={},
    obs={},
    paired={},
    start_time=None,
    end_time=None,
    download_maps=True,
    output_dir=None,
    debug=False,
)

In [3]:
an.read_control()
an

analysis(
    control='control_idealized.yaml',
    control_dict=...,
    models={},
    obs={},
    paired={},
    start_time=Timestamp('2019-09-09 00:00:00'),
    end_time=Timestamp('2019-09-10 00:00:00'),
    download_maps=True,
    output_dir='./output/idealized',
    debug=True,
)

## Generate data

### Model

In [4]:
np.random.seed(42)

control = an.control_dict

nlat = 100
nlon = 200

lon = np.linspace(-172, -66, nlon)
lat = np.linspace(18, 72, nlat)

lat_da = xr.DataArray(lat, attrs={'longname': 'latitude', 'units': 'degN'})
lon_da = xr.DataArray(lon, attrs={'longname': 'longitude', 'units': 'degE'})

time = pd.date_range(control['analysis']['start_time'], control['analysis']['end_time'], freq="3H")
ntime = time.size

# Generate dataset
field_names = control['model']['test_model']['variables'].keys()
ds_dict = dict()
for field_name in field_names:
    units = control['model']['test_model']['variables'][field_name]['units']
    data = np.random.rand(ntime, nlat, nlon)
    da = xr.DataArray(
        data,
        coords=[time, lat_da, lon_da],
        dims=['time', 'lat', 'lon'],
        attrs={'units': units},
    )
    ds_dict[field_name] = da
ds = xr.Dataset(ds_dict).expand_dims("z", axis=1)
ds

<xarray.Dataset>
Dimensions:  (time: 9, lat: 100, lon: 200, z: 1)
Coordinates:
  * time     (time) datetime64[ns] 2019-09-09 2019-09-09T03:00:00 ... 2019-09-10
  * lat      (lat) float64 18.0 18.55 19.09 19.64 ... 70.36 70.91 71.45 72.0
  * lon      (lon) float64 -172.0 -171.5 -170.9 -170.4 ... -67.07 -66.53 -66.0
Dimensions without coordinates: z
Data variables:
    A        (time, z, lat, lon) float64 0.3745 0.9507 0.732 ... 0.388 0.481
    B        (time, z, lat, lon) float64 0.2141 0.7517 0.2915 ... 0.9236 0.3339

In [5]:
ds.to_netcdf(control['model']['test_model']['files'])

### Obs

In [6]:
# Generate positions
n = 400
lats = np.random.uniform(lat[0], lat[-1], n)[np.newaxis, :]
lons = np.random.uniform(lon[0], lon[-1], n)[np.newaxis, :]
siteid = np.arange(n)[np.newaxis, :].astype(str)

# Generate dataset
field_names = control['model']['test_model']['variables'].keys()
ds_dict = dict()
for field_name0 in field_names:
    field_name = control['model']['test_model']['mapping']['test_obs'][field_name0]
    units = control['model']['test_model']['variables'][field_name0]['units']
    values = np.random.rand(ntime, n)[:, np.newaxis]
    da = xr.DataArray(
        values,
        coords={
            "time": ("time", time),
            "x": ("x", np.arange(n)),
            "latitude": (("y", "x"), lats, lat_da.attrs),
            "longitude": (("y", "x"), lons, lon_da.attrs),
            "siteid": (("y", "x"), siteid),
        },
        dims=("time", "y", "x"),
        attrs={'units': units},
    )
    ds_dict[field_name] = da
ds = xr.Dataset(ds_dict)
ds

<xarray.Dataset>
Dimensions:    (time: 9, x: 400, y: 1)
Coordinates:
  * time       (time) datetime64[ns] 2019-09-09 ... 2019-09-10
  * x          (x) int32 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
    latitude   (y, x) float64 39.65 23.36 53.61 61.53 ... 49.52 55.98 42.36 54.8
    longitude  (y, x) float64 -160.0 -171.3 -123.3 ... -77.83 -76.96 -80.4
    siteid     (y, x) <U11 '0' '1' '2' '3' '4' ... '395' '396' '397' '398' '399'
Dimensions without coordinates: y
Data variables:
    A_obs      (time, y, x) float64 0.9726 0.7788 0.1493 ... 0.31 0.00943 0.1703
    B_obs      (time, y, x) float64 0.1804 0.6997 0.8791 ... 0.242 0.3573 0.7642

In [7]:
ds.to_netcdf(control['obs']['test_obs']['filename'])

## Load data

In [8]:
an.open_models()

test_model
{'files': 'test_model.nc', 'mod_type': 'random', 'variables': {'A': {'units': 'Units of A', 'unit_scale': 1, 'unit_scale_method': '*'}, 'B': {'units': 'Units of B', 'unit_scale': 1, 'unit_scale_method': '*'}}, 'mapping': {'test_obs': {'A': 'A_obs', 'B': 'B_obs'}}, 'projection': 'None'}
test_model.nc


In [9]:
an.models['test_model'].obj

<xarray.Dataset>
Dimensions:  (time: 9, lat: 100, lon: 200, z: 1)
Coordinates:
  * time     (time) datetime64[ns] 2019-09-09 2019-09-09T03:00:00 ... 2019-09-10
  * lat      (lat) float64 18.0 18.55 19.09 19.64 ... 70.36 70.91 71.45 72.0
  * lon      (lon) float64 -172.0 -171.5 -170.9 -170.4 ... -67.07 -66.53 -66.0
Dimensions without coordinates: z
Data variables:
    A        (time, z, lat, lon) float64 0.3745 0.9507 0.732 ... 0.388 0.481
    B        (time, z, lat, lon) float64 0.2141 0.7517 0.2915 ... 0.9236 0.3339

In [10]:
an.open_obs()

In [11]:
an.obs['test_obs'].obj

<xarray.Dataset>
Dimensions:    (time: 9, x: 400, y: 1)
Coordinates:
  * time       (time) datetime64[ns] 2019-09-09 ... 2019-09-10
  * x          (x) int32 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
    latitude   (y, x) float64 ...
    longitude  (y, x) float64 ...
    siteid     (y, x) object ...
Dimensions without coordinates: y
Data variables:
    A_obs      (time, y, x) float64 ...
    B_obs      (time, y, x) float64 ...

In [12]:
an.pair_data()

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.2s
[########################################] | 100% Completed |  0.3s
[########################################] | 100% Completed |  0.4s


In [13]:
an.paired

{'test_obs_test_model': pair(
     type='pt_sfc',
     radius_of_influence=1000000.0,
     obs='test_obs',
     model='test_model',
     model_vars=['A', 'B'],
     obs_vars=['A_obs', 'B_obs'],
     filename='test_obs_test_model.nc',
 )}

In [14]:
an.paired['test_obs_test_model'].obj.to_dataframe()

A_obs     B_obs         A         B   latitude  \
x   time                                                                     
0   2019-09-09 00:00:00  0.972629  0.180360  0.801352  0.035132  39.647508   
    2019-09-09 03:00:00  0.844250  0.191834  0.358484  0.327631  39.647508   
    2019-09-09 06:00:00  0.519456  0.785643  0.384771  0.537533  39.647508   
    2019-09-09 09:00:00  0.370871  0.684209  0.181457  0.219687  39.647508   
    2019-09-09 12:00:00  0.650170  0.572420  0.929427  0.183187  39.647508   
...                           ...       ...       ...       ...        ...   
399 2019-09-09 12:00:00  0.985851  0.497445  0.421840  0.178392  24.112507   
    2019-09-09 15:00:00  0.044695  0.888375  0.322952  0.801574  24.112507   
    2019-09-09 18:00:00  0.036533  0.987415  0.877293  0.770136  24.112507   
    2019-09-09 21:00:00  0.192230  0.222583  0.897503  0.594696  24.112507   
    2019-09-10 00:00:00  0.846886  0.659962  0.991461  0.328304  24.112507   

                          longitude siteid  
x   time                                    
0   2019-09-09 00:00:00 -160.042231      0  
    2019-09-09 03:00:00 -160.042231      0  
    2019-09-09 06:00:00 -160.042231      0  
    2019-09-09 09:00:00 -160.042231      0  
    2019-09-09 12:00:00 -160.042231      0  
...                             ...    ...  
399 2019-09-09 12:00:00 -131.133457     99  
    2019-09-09 15:00:00 -131.133457     99  
    2019-09-09 18:00:00 -131.133457     99  
    2019-09-09 21:00:00 -131.133457     99  
    2019-09-10 00:00:00 -131.133457     99  

[3600 rows x 7 columns]

## Plot

In [15]:
an.plotting()

TypeError: Level type mismatch: 2019-09-09 00:00:00